In [ ]:
import h5py
import math
import array
from numpy import *
import numpy as np
from pycbc.types import TimeSeries
from pycbc.waveform import get_td_waveform
from pycbc import types, waveform
from scipy import interpolate
from scipy.interpolate import interp1d
from lal import MSUN_SI, MTSUN_SI, G_SI, PC_SI, C_SI, PI
from tqdm import tqdm

### Hinder+ modified all 20 simulations SEOBNRv4 model, full frequency range

In [1]:
#edit to provide location of the fit files

g=open('/home/croyt/GW_codes/TD_model/HMmodel/tshift_H+modified_20hyb_Feb16.txt',"r")
lines=g.readlines() 
A=[] 
for x in lines: 
    A.append(float(x.split()[1])) 
g.close()

def tshift_Hinsp(q,e,l): 
    return A[0] + A[1]*q + A[2]*q**2 + A[3]*e + A[4]*e**2 + A[5]*e**3 + A[6]*e*q + A[7]*(e**2)*q + A[8]*e*(q**2) + A[9]*(e**2)*(q**2) + A[10]*(e**3)*q + A[11]*e*q*math.cos(l + A[12]) + A[13]*(e**2)*(q**2)*math.cos(e*l + A[14]) + A[15]*(e**3)*q*math.cos(l + A[16]) + A[17]*e*(q**2)*math.cos(l + A[18])

g=open('/home/croyt/GW_codes/TD_model/HMmodel/tamp_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
B=[] 
for x in lines: 
    B.append(float(x.split()[1])) 
g.close()

def tamp_Hinsp(eta,e,l): 
    return B[0] + B[1]*eta + B[2]*eta**2 + B[3]*e + B[4]*e**2 + B[5]*e**3 + B[6]*e*eta + B[7]*(e**2)*eta + B[8]*e*(eta**2) + B[9]*(e**2)*(eta**2) + B[10]*(e**3)*eta + B[11]*e*eta*math.cos(l + B[12]) + B[13]*(e**2)*(eta**2)*math.cos(l + B[14]) + B[15]*(e**3)*eta*math.cos(e*l + B[16])

g=open('/home/croyt/GW_codes/TD_model/HMmodel/tfreq_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
C=[] 
for x in lines: 
    C.append(float(x.split()[1])) 
g.close()

def tfreq_Hinsp(eta,e,l): 
    return C[0] + C[1]*eta + C[2]*eta**2 + C[3]*e + C[4]*e**2 + C[5]*e**3 + C[6]*e*eta + C[7]*(e**2)*eta + C[8]*e*(eta**2) + C[9]*(e**2)*(eta**2) + C[10]*(e**3)*eta + C[11]*e*eta*math.cos(l + C[12]) + C[13]*(e**2)*(eta**2)*math.cos(l + C[14]) + C[15]*(e**3)*eta*math.cos(e*l + C[16]) + C[17]*(e**3)*(eta**2)*math.cos(e*l + C[18])

In [5]:
def xi(x):
    return x**(3/2)

def xconv(f,M):
    return (PI*M*MTSUN_SI*f)**(2/3)  #22 mode conversion formula

def fconv(x,M):
    return x**(3/2)/(PI*M*MTSUN_SI)  #22 mode conversion


In [32]:
def eccmodel(Mass,q0,e0,l0,fmin,inclination=0,d=1,delta_t=1/1024,modes=[[2,2],[3,3],[4,4],[5,5]]):
    
    ell = []
    numrows = len(modes)    # no. of rows 
    numcols = len(modes[0]) # no. of columns 
    for i in range(0,numrows):
        l = modes[i][0]
        m = modes[i][1]
        ell.append(l)
    angle = inclination
    waveform = {}
    count = 0
    el = 2  
    if el in ell:
        mode_data = {}
        #hp22, hc22, t22 = model_22(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        #return hp22, hc22, t22
        mode_data['hp'], mode_data['hc'], mode_data['t'] = model_22(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l2_m2'] = mode_data
        count  = count + 1
        #print('22 mode')
    el = 3 
    if el in ell:
        mode_data = {}
        #hp33, hc33, t33 = model_33(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        mode_data['hp'], mode_data['hc'], mode_data['t'] = model_33(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l3_m3'] = mode_data
        count  = count + 1
        #print('33 mode')
    el = 4 
    if el in ell:
        mode_data = {}
        #hp44, hc44, t44 = model_44(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        mode_data['hp'], mode_data['hc'], mode_data['t'] = model_44(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l4_m4'] = mode_data
        count  = count + 1
        #print('44 mode')
    el = 5 
    if el in ell:
        mode_data = {}
        #hp55, hc55, t55 = model_55(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        mode_data['hp'], mode_data['hc'], mode_data['t'] = model_55(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l5_m5'] = mode_data
        count  = count + 1
        #print('55 mode')
    
    #print(count)
    len_max_mode = '0'
    len_max = 0
    for mode in waveform.keys():
        if len(waveform[mode]['t'])>len_max:
            len_max_mode = mode
            len_max = len(waveform[mode]['t'])
    
    for mode in waveform.keys():
        if mode != len_max_mode:
            waveform[mode]['hp'].resize(len_max)
            waveform[mode]['hc'].resize(len_max)
    
    hp=0
    hc=0
    time=waveform[len_max_mode]['t']
    for mode in waveform.keys():
        hp = hp + waveform[mode]['hp']
        hc = hc + waveform[mode]['hp']
        
    hplus = TimeSeries(hp,delta_t,epoch=time[0])
    hcross = TimeSeries(hc,delta_t,epoch=time[0])
    
    return hplus, hcross
    #return hp, hc, time
        

In [ ]:
def sph_harmonics(inc,ell):
    L=ell
    #inc = 10
    theta = inc
    for l in range(L,L+1):

        for m in range(-l,l+1):
            dlm = 0;
            k1 = max([0, m-2]);
            k2 = min([l+m, l-2]);

            #if(m==l or m==l-1):
            for k in range(k1,k2+1):
                A = []; B = []; cosTerm = []; sinTerm = []; dlmTmp = [];

                A = (-1)**k*math.sqrt(math.factorial(l+m)*math.factorial(l-m)*math.factorial(l+2)*math.factorial(l-2));
                B = math.factorial(k)*math.factorial(k-m+2)*math.factorial(l+m-k)*math.factorial(l-k-2);

                cosTerm = pow(math.cos(theta/2), 2*l+m-2*k-2);
                sinTerm = pow(math.sin(theta/2), 2*k-m+2);

                dlmTmp = (A/B)*cosTerm*sinTerm;
                dlm = dlm+dlmTmp

            Ylm = math.sqrt((2*l+1)/(4*math.pi))*dlm
            #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
            if m==ell:
                #globals()['sph' + str(l) + str(m)] = Ylm
                #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
                sphlm = Ylm
            elif m==-ell:
                #globals()['sph' + str(l) + '_' + str(abs(m))] = Ylm
                #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
                sphl_m = Ylm
            else:
                continue
    return sphlm, sphl_m

In [ ]:
def PNparams(M,q,d,f_low,e0,delta_t):
    M2=M/(1+q)
    M1=M2*q
    hpVec_PN, hcVec_PN = get_td_waveform(approximant='EccentricTD', mass1=M1, mass2=M2, 
                             delta_t=delta_t, 
                             f_lower=f_low, 
                             eccentricity=e0,
                             distance=d)
    
    modetopolfac=4*(5/(64*np.pi))**(1/2)  #conversion factor between mode amplitude and strain amplitude
                                          # check 0704.3764 equation 7 for details
    M_SI = M * MSUN_SI
    D_SI = 10**6 * PC_SI * d
    phase_EccTD = waveform.utils.phase_from_polarizations(hpVec_PN/(modetopolfac*(G_SI*M_SI/D_SI/C_SI/C_SI)), hcVec_PN/(modetopolfac*(G_SI*M_SI/D_SI/C_SI/C_SI)))
    tVec_PN=hpVec_PN.sample_times/(M*MTSUN_SI)
    
    return phase_EccTD, tVec_PN

In [ ]:
%run GW_functions_model.ipynb

model_22(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    m = total mass in units of solar mass
    q0 = mass ratio
    e0 = initial eccentricity
    l0 = initial mean anomaly (-PI to PI)
    fmin = initial frequency in units of Hz
    angle = orbit inclination
    d = distanve in units of MPc
    delta_t = sampling rate

### $(\ell, m) = (2, 2)$ mode waveform

In [2]:
def model_22(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    #m = total mass in units of solar mass
    #q0 = mass ratio
    #e0 = initial eccentricity
    #l0 = initial mean anomaly (-PI to PI)
    #fmin = initial frequency in units of Hz
    #angle = orbit inclination
    #d = distanve in units of MPc
    #delta_t = sampling rate

    M=m
    M1 = q0*M/(1+q0)
    M2 = M/(1+q0)
    eta = q0/(1+q0)**2
    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    mode2polfac=4*(5/(64*np.pi))**(1/2)

    hp, hc = get_td_waveform(approximant='EccentricTD', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin, eccentricity=e0)
    sp, sc = get_td_waveform(approximant='SEOBNRv4', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin)

    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI
    tmin = max(hp.sample_times[0]-tshift,sp.sample_times[0])

    #circular IMR
    sp_intrp = interp1d(sp.sample_times, sp, kind='cubic',fill_value='extrapolate')
    sc_intrp = interp1d(sc.sample_times, sc, kind='cubic',fill_value='extrapolate')
    tImr_intrp=np.arange(tmin, sp.sample_times[-1], delta_t)
    sp_intrp = sp_intrp(tImr_intrp)
    sc_intrp = sc_intrp(tImr_intrp)
    tImr = tImr_intrp
    hpImr = sp_intrp/mode2polfac
    hcImr = sc_intrp/mode2polfac
    h22Imr = hpImr + 1j*hcImr


    "time shift"
    "Hinder+ inspired"
    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI

    # interpolation EccentricTD
    hp_intrp = interp1d(hp.sample_times-tshift, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(hc.sample_times-tshift, hc, kind='cubic',fill_value='extrapolate')
    # new (unifrom) sampling rate (for interpolation and further processing)
    tEcc_intrp=np.arange(tmin, hp.sample_times[-1]-tshift, delta_t)
    hp_intrp = hp_intrp(tEcc_intrp)
    hc_intrp = hc_intrp(tEcc_intrp)
    tEcc = tEcc_intrp
    hpEcc = hp_intrp/mode2polfac
    hcEcc = hc_intrp/mode2polfac
    h22Ecc = hpEcc + 1j*hcEcc

    phaseEcc = np.unwrap(np.angle(h22Ecc)*2)/2
    phaseImr = np.unwrap(np.angle(h22Imr)*2)/2
    dphase = phaseEcc[0] - phaseImr[0]
    hp_new = real(h22Ecc * exp(-1j * dphase))
    hc_new = imag(h22Ecc * exp(-1j * dphase))

    phase_new = np.unwrap(np.angle(hp_new+1j*hc_new)*2)/2

    phaseEcc = phase_new
    h22Ecc_new = hp_new+1j*hc_new

    if(tamp_Hinsp(eta,e0,l0)>=0.0):
        print('t_amp>0')

    "Hinder+ inspired"
    arg = np.argmin(abs(tEcc - tamp_Hinsp(eta,e0,l0)*M*MTSUN_SI))
    Idxjoin = arg 

    "Hinder+ inspired"
    t_amp = tEcc[Idxjoin] - 500*M*MTSUN_SI
    idxstr = np.argmin(abs(tEcc-t_amp))

    "amplitude model"
    amp = []
    count = 0
    length = Idxjoin - idxstr 

    for i in range(idxstr,Idxjoin):
        amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
        count = count+1

    t_model = np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))
    h22amp = np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
    h22amp_model = np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))

    omegaEcc = (M*MTSUN_SI/delta_t)*(np.gradient(phaseEcc))
    omegaImr = (M*MTSUN_SI/delta_t)*(np.gradient(phaseImr))

    if(tfreq_Hinsp(eta,e0,l0)>=0.0):
        print('t_freq>0')

    "Hider+ inspired"
    tjoin0 = tfreq_Hinsp(eta,e0,l0)
    tjoin = tjoin0 * M * MTSUN_SI
    fjoin = np.argmin(abs(tEcc-tjoin))

    "frequency model"
    tstop = min(tEcc[-1],-30*M*MTSUN_SI)
    lst = np.argmin(abs(tEcc-tstop))

    indx = lst - fjoin
    a0 = []
    n = indx - 1 
    k = 0
    for i in range(fjoin,fjoin+indx):
        a0.append(((n-k)*omegaEcc[i]+k*omegaImr[i])/n)
        k = k+1

    f1 = np.concatenate((omegaEcc[0:fjoin],a0))
    frequency_model = np.concatenate((f1,omegaImr[fjoin+indx:len(omegaImr)]))
    phase_f_model = np.cumsum(frequency_model)/(M*MTSUN_SI/delta_t)


    hp_f_model = h22amp_model * np.cos(phase_f_model)
    hc_f_model = h22amp_model * np.sin(phase_f_model)
    
    ht = (mode2polfac/4)*(((1+math.cos(angle))**2 * (hp_f_model - 1j*hc_f_model)) + ((1-math.cos(angle))**2 * (hp_f_model + 1j*hc_f_model)))
    hplus = np.real(ht)
    hcross = np.imag(ht)
    
    return np.array(hplus), np.array(hcross), np.array(t_model)

### $(\ell, m) = (3, 3)$ mode waveform

In [ ]:
def inspiral_33(M0,q,e0,l0,flow,inc,d0,delta_t):
    eta=neu=nu=q/(1+q)**2 
    #xref=0.075  #reference frequency
    #xlow=0.045  #initial frequency
    G=c=M=d=1
    M2=M/(1+q)
    M1=M2*q
    Delta=math.sqrt(1-(4*neu))
    eta=nu=neu
    gamma=EulerGamma= 0.577215664901

    conv=M*MTSUN_SI              #total mass (needed to convert PN time t to t/M)
    M_SI = M * MSUN_SI
    D_SI = 10**6 * PC_SI * d

    xlow = ((M0*MTSUN_SI*math.pi*flow)**(2/3))
    f_low =(xlow**(3/2)/(M*MTSUN_SI*math.pi)) #Lower frequency of EccTD wf for 1Msun system
    #f_ref=freq(xref)            #Ref frequency of EccTD wf
    ####################################################
    
    #if l0<0:
        #l0 = l0 + 2*np.pi
    
    #Calculating ecc at xlow for generating EccTD wf-->eccentricity evolution Eq. (4.17a,b) Moore et. al.(2016)

    #%run GW_functions_model.ipynb          #Notebook with all PN evolution equations   
    x=xlow                          #x=point where eccentricity is required
    v=math.sqrt(x)
    #vref=math.sqrt(xref)
    xie=v**3
    #xiref=vref**3

    #Gives e at xie with b.c.--> e=eref when xie=xiref
    #e=eref*(xiref/xie)**(19/18)*epsilon(xie, eta)/epsilon(xiref, eta)  #eccentricity evolution 
    
    if delta_t>=1/2**14:
        del_t = 1/2**14
    elif delta_t<1/2**14 and delta_t>=1/2**16:
        del_t = 1/2**16
    elif delta_t<1/2**16 and delta_t>=1/2**18:
        del_t = 1/2**18
    else:
        del_t = 1/2**20
    
    phase_EccTD, tVec_PN = PNparams(M,q,d,f_low,e0,del_t)
    
    tC_NR = 0
    #xi_ref=xref**(3/2)
    #v_ref=xi_ref**(1/3)
    #input initial params
    x0=xlow
    xi0=x0**(3/2)
    v0=xi0**(1/3)
    #e0=(eref*(xi_ref/xi0)**(19/18))*(epsilon(xi0, eta)/epsilon(xi_ref, eta))
    #l0=mean_anomaly(xi_ref, xi0, lref, eta, e0)
    theta=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN)**(-1/8)
    theta0=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN[0])**(-1/8)
    fVec=x_from_t(theta, theta0, e0, M, eta)
    
    plotIdx2=np.nonzero(fVec>=0)
    fVec=fVec[plotIdx2]
    
    xiVec=(np.pi*M*fVec)
    xVec=xiVec**(2/3)
    vVec=xiVec**(1/3)
    xband=np.where(xVec<=1/6)
    xVec = xVec[xband]
    maxPNidx=len(xVec)
    tVec_PN=tVec_PN[:maxPNidx]
    
    
    lp=3
    mp=3
    
    j=0
    #omega=[]
    h33=[]
    h3_3=[]
    for i in xVec:
        v=math.sqrt(i)
        v0=math.sqrt(x0)
        xie=v**3
        xi0=v0**3
        l=mean_anomaly(xie, xi0, l0, eta, e0)
        e=e0*(xi0/xie)**(19/18)*epsilon(xie, eta)/epsilon(xi0, eta)
        #psi=phase_TT2(v, v0, phiC, eta)+W(xie, xi0, eta, e0, l)
        psi=phase_EccTD[j]
        j=j+1
        xi=l   #use xi for amplitude (xie is being used for v**3)
        x=i
        h=amplitude_33(xi,x,nu,e,Delta)  #check GW_functions.ipynb for the respective mode amplitude functions 
                                   #and the required function arguements
        hlm=8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,-1)*mp*psi/2))/d
        hl_m = 8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,+1)*mp*psi/2))/d

        h33.append(hlm)
        h3_3.append(hl_m)
        
    conv_t = M0*MTSUN_SI
    conv_h = G_SI*M0*MSUN_SI/(10**6 * PC_SI * d0)/C_SI/C_SI

    sph33, sph3_3 = sph_harmonics(inc,lp)
        
    h = np.multiply(h33,sph33) + np.multiply(h3_3,sph3_3)
    hp = np.real(h)
    hc = np.imag(h)
    time = tVec_PN - tVec_PN[-1]
        
    hp = np.array(hp) * conv_h
    hc = np.array(hc) * conv_h
    time = tVec_PN * conv_t
    
    hp_intrp = interp1d(time, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(time, hc, kind='cubic',fill_value='extrapolate')
    t_intrp=np.arange(time[0], time[-1], delta_t)
    hp_intrp = hp_intrp(t_intrp)
    hc_intrp = hc_intrp(t_intrp)
    
    #hpTS = TimeSeries(hp_intrp,delta_t)
    #hcTS = TimeSeries(hc_intrp,delta_t)
    
    return np.array(hp_intrp), np.array(hc_intrp), np.array(t_intrp)

In [ ]:
def model_33(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    #m = total mass in units of solar mass
    #q0 = mass ratio
    #e0 = initial eccentricity
    #l0 = initial mean anomaly (-PI to PI)
    #fmin = initial frequency in units of Hz
    #angle = orbit inclination
    #d = distanve in units of MPc
    #delta_t = sampling rate
    
    M=m
    M1 = q0*M/(1+q0)
    M2 = M/(1+q0)
    eta = q0/(1+q0)**2
    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI


    hp, hc, tinsp = inspiral_33(M,q0,e0,l0,fmin,angle,d,delta_t)
    sp, sc = get_td_waveform(approximant='SEOBNRv4HM', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin, mode_array=[[3,3]],inclination=angle)


    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI
    tmin = max(tinsp[0]-tshift,sp.sample_times[0])

    #circular IMR
    sp_intrp = interp1d(sp.sample_times, sp, kind='cubic',fill_value='extrapolate')
    sc_intrp = interp1d(sc.sample_times, sc, kind='cubic',fill_value='extrapolate')
    tImr_intrp=np.arange(tmin, sp.sample_times[-1], delta_t)
    sp_intrp = sp_intrp(tImr_intrp)
    sc_intrp = sc_intrp(tImr_intrp)
    tImr = tImr_intrp
    hpImr = sp_intrp
    hcImr = sc_intrp
    h22Imr = hpImr + 1j*hcImr


    "time shift"
    "Hinder+ inspired"
    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI

    # interpolation EccentricTD
    hp_intrp = interp1d(tinsp-tshift, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(tinsp-tshift, hc, kind='cubic',fill_value='extrapolate')
    # new (unifrom) sampling rate (for interpolation and further processing)
    tEcc_intrp=np.arange(tmin, tinsp[-1]-tshift, delta_t)
    hp_intrp = hp_intrp(tEcc_intrp)
    hc_intrp = hc_intrp(tEcc_intrp)
    tEcc = tEcc_intrp
    hpEcc = hp_intrp
    hcEcc = hc_intrp
    h22Ecc = hpEcc + 1j*hcEcc

    phaseEcc = np.unwrap(np.angle(h22Ecc)*2)/2
    phaseImr = -np.unwrap(np.angle(h22Imr)*2)/2
    dphase = phaseEcc[0] - phaseImr[0]
    hp_new = real(h22Ecc * exp(-1j * dphase))
    hc_new = imag(h22Ecc * exp(-1j * dphase))

    phase_new = np.unwrap(np.angle(hp_new+1j*hc_new)*2)/2

    phaseEcc = phase_new
    h22Ecc_new = hp_new+1j*hc_new

    if(tamp_Hinsp(eta,e0,l0)>=0.0):
        print('t_amp>0')

    "Hinder+ inspired"
    arg = np.argmin(abs(tEcc - tamp_Hinsp(eta,e0,l0)*M*MTSUN_SI))
    Idxjoin = arg 

    "Hinder+ inspired"
    t_amp = tEcc[Idxjoin] - 500*M*MTSUN_SI
    idxstr = np.argmin(abs(tEcc-t_amp))

    "amplitude model"
    amp = []
    count = 0
    length = Idxjoin - idxstr 

    for i in range(idxstr,Idxjoin):
        amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
        count = count+1

    t_model = np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))
    h22amp = np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
    h22amp_model = np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))

    omegaEcc = (M*MTSUN_SI/delta_t)*(np.gradient(phaseEcc))
    omegaImr = (M*MTSUN_SI/delta_t)*(np.gradient(phaseImr))

    if(tfreq_Hinsp(eta,e0,l0)>=0.0):
        print('t_freq>0')

    "Hider+ inspired"
    tjoin0 = tfreq_Hinsp(eta,e0,l0)
    tjoin = tjoin0 * M * MTSUN_SI
    fjoin = np.argmin(abs(tEcc-tjoin))

    "frequency model"
    tstop = min(tEcc[-1],-30*M*MTSUN_SI)
    lst = np.argmin(abs(tEcc-tstop))

    indx = lst - fjoin
    a0 = []
    n = indx - 1 
    k = 0
    for i in range(fjoin,fjoin+indx):
        a0.append(((n-k)*omegaEcc[i]+k*omegaImr[i])/n)
        k = k+1

    f1 = np.concatenate((omegaEcc[0:fjoin],a0))
    frequency_model = np.concatenate((f1,omegaImr[fjoin+indx:len(omegaImr)]))
    phase_f_model = np.cumsum(frequency_model)/(M*MTSUN_SI/delta_t)

    hp_f_model = h22amp_model * np.cos(phase_f_model)
    hc_f_model = h22amp_model * np.sin(phase_f_model)
    
    return np.array(hp_f_model), np.array(hc_f_model), np.array(t_model)

### $(\ell, m) = (4, 4)$ mode waveform

In [ ]:
def inspiral_44(M0,q,e0,l0,flow,inc,d0,delta_t):
    eta=neu=nu=q/(1+q)**2 
    #xref=0.075  #reference frequency
    #xlow=0.045  #initial frequency
    G=c=M=d=1
    M2=M/(1+q)
    M1=M2*q
    Delta=math.sqrt(1-(4*neu))
    eta=nu=neu
    gamma=EulerGamma= 0.577215664901

    conv=M*MTSUN_SI              #total mass (needed to convert PN time t to t/M)
    M_SI = M * MSUN_SI
    D_SI = 10**6 * PC_SI * d

    xlow = ((M0*MTSUN_SI*math.pi*flow)**(2/3))
    f_low =(xlow**(3/2)/(M*MTSUN_SI*math.pi)) #Lower frequency of EccTD wf which is in our hands
    #f_ref=freq(xref)            #Ref frequency of EccTD wfD wf which is in our hands
    ####################################################
    
    #if l0<0:
        #l0 = l0 + 2*np.pi
    
    #Calculating ecc at xlow for generating EccTD wf-->eccentricity evolution Eq. (4.17a,b) Moore et. al.(2016)

    #%run GW_functions_model.ipynb         #Notebook with all PN evolution equations   
    x=xlow                          #x=point where eccentricity is required
    v=math.sqrt(x)
    #vref=math.sqrt(xref)
    xie=v**3
    #xiref=vref**3

    #Gives e at xie with b.c.--> e=eref when xie=xiref
    #e=eref*(xiref/xie)**(19/18)*epsilon(xie, eta)/epsilon(xiref, eta)  #eccentricity evolution 
    
    if delta_t>=1/2**14:
        del_t = 1/2**14
    elif delta_t<1/2**14 and delta_t>=1/2**16:
        del_t = 1/2**16
    elif delta_t<1/2**16 and delta_t>=1/2**18:
        del_t = 1/2**18
    else:
        del_t = 1/2**20
    
    phase_EccTD, tVec_PN = PNparams(M,q,d,f_low,e0,del_t)
    
    tC_NR = 0
    #xi_ref=xref**(3/2)
    #v_ref=xi_ref**(1/3)
    #input initial params
    x0=xlow
    xi0=x0**(3/2)
    v0=xi0**(1/3)
    #e0=(eref*(xi_ref/xi0)**(19/18))*(epsilon(xi0, eta)/epsilon(xi_ref, eta))
    #l0=mean_anomaly(xi_ref, xi0, lref, eta, e0)
    theta=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN)**(-1/8)
    theta0=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN[0])**(-1/8)
    fVec=x_from_t(theta, theta0, e0, M, eta)
    
    plotIdx2=np.nonzero(fVec>=0)
    fVec=fVec[plotIdx2]
    
    xiVec=(np.pi*M*fVec)
    xVec=xiVec**(2/3)
    vVec=xiVec**(1/3)
    xband=np.where(xVec<=1/6)
    xVec = xVec[xband]
    maxPNidx=len(xVec)
    tVec_PN=tVec_PN[:maxPNidx]
    
    
    lp=4
    mp=4
    
    j=0
    #omega=[]
    h44=[]
    h4_4=[]
    for i in xVec:
        v=math.sqrt(i)
        v0=math.sqrt(x0)
        xie=v**3
        xi0=v0**3
        l=mean_anomaly(xie, xi0, l0, eta, e0)
        e=e0*(xi0/xie)**(19/18)*epsilon(xie, eta)/epsilon(xi0, eta)
        #psi=phase_TT2(v, v0, phiC, eta)+W(xie, xi0, eta, e0, l)
        psi=phase_EccTD[j]
        j=j+1
        xi=l   #use xi for amplitude (xie is being used for v**3)
        x=i
        h=amplitude_44(xi,x,nu,e,Delta)  #check GW_functions.ipynb for the respective mode amplitude functions 
                                   #and the required function arguements
        hlm=8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,-1)*mp*psi/2))/d
        hl_m = 8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,+1)*mp*psi/2))/d

        h44.append(hlm)
        h4_4.append(hl_m)
        
    conv_t = M0*MTSUN_SI
    conv_h = G_SI*M0*MSUN_SI/(10**6 * PC_SI * d0)/C_SI/C_SI

    sph44, sph4_4 = sph_harmonics(inc,lp)
        
    h = np.multiply(h44,sph44) + np.multiply(h4_4,sph4_4)
    hp = np.real(h)
    hc = np.imag(h)
    time = tVec_PN - tVec_PN[-1]
        
    hp = np.array(hp) * conv_h
    hc = np.array(hc) * conv_h
    time = tVec_PN * conv_t
    
    hp_intrp = interp1d(time, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(time, hc, kind='cubic',fill_value='extrapolate')
    t_intrp=np.arange(time[0], time[-1], delta_t)
    hp_intrp = hp_intrp(t_intrp)
    hc_intrp = hc_intrp(t_intrp)
    
    return np.array(hp_intrp), np.array(hc_intrp), np.array(t_intrp)

In [ ]:
def model_44(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    #m = total mass in units of solar mass
    #q0 = mass ratio
    #e0 = initial eccentricity
    #l0 = initial mean anomaly (-PI to PI)
    #fmin = initial frequency in units of Hz
    #angle = orbit inclination
    #d = distanve in units of MPc
    #delta_t = sampling rate
    
    M=m
    M1 = q0*M/(1+q0)
    M2 = M/(1+q0)
    eta = q0/(1+q0)**2
    #q0 = q
    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    #mode2polfac=4*(5/(64*np.pi))**(1/2)

    hp, hc, tinsp = inspiral_44(M,q0,e0,l0,fmin,angle,d,delta_t)
    sp, sc = get_td_waveform(approximant='SEOBNRv4HM', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin, mode_array=[[4,4]],inclination=angle)

    #plt.plot(tinsp,hp)
    #plt.plot(sp.sample_times,sp)
    #'''
    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI
    tmin = max(tinsp[0]-tshift,sp.sample_times[0])

    #circular IMR
    sp_intrp = interp1d(sp.sample_times, sp, kind='cubic',fill_value='extrapolate')
    sc_intrp = interp1d(sc.sample_times, sc, kind='cubic',fill_value='extrapolate')
    tImr_intrp=np.arange(tmin, sp.sample_times[-1], delta_t)
    sp_intrp = sp_intrp(tImr_intrp)
    sc_intrp = sc_intrp(tImr_intrp)
    tImr = tImr_intrp
    hpImr = sp_intrp
    hcImr = sc_intrp
    h22Imr = hpImr + 1j*hcImr


    "time shift"
    "Hinder+ inspired"
    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI

    # interpolation EccentricTD
    hp_intrp = interp1d(tinsp-tshift, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(tinsp-tshift, hc, kind='cubic',fill_value='extrapolate')
    # new (unifrom) sampling rate (for interpolation and further processing)
    tEcc_intrp=np.arange(tmin, tinsp[-1]-tshift, delta_t)
    hp_intrp = hp_intrp(tEcc_intrp)
    hc_intrp = hc_intrp(tEcc_intrp)
    tEcc = tEcc_intrp
    hpEcc = hp_intrp
    hcEcc = hc_intrp
    h22Ecc = hpEcc + 1j*hcEcc

    phaseEcc = np.unwrap(np.angle(h22Ecc)*2)/2
    phaseImr = -np.unwrap(np.angle(h22Imr)*2)/2
    dphase = phaseEcc[0] - phaseImr[0]
    hp_new = real(h22Ecc * exp(-1j * dphase))
    hc_new = imag(h22Ecc * exp(-1j * dphase))

    phase_new = np.unwrap(np.angle(hp_new+1j*hc_new)*2)/2

    phaseEcc = phase_new
    h22Ecc_new = hp_new+1j*hc_new

    if(tamp_Hinsp(eta,e0,l0)>=0.0):
        print('t_amp>0')

    "Hinder+ inspired"
    arg = np.argmin(abs(tEcc - tamp_Hinsp(eta,e0,l0)*M*MTSUN_SI))
    Idxjoin = arg 

    "Hinder+ inspired"
    t_amp = tEcc[Idxjoin] - 500*M*MTSUN_SI
    idxstr = np.argmin(abs(tEcc-t_amp))

    "amplitude model"
    amp = []
    count = 0
    length = Idxjoin - idxstr 

    for i in range(idxstr,Idxjoin):
        amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
        count = count+1

    t_model = np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))
    h22amp = np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
    h22amp_model = np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))

    omegaEcc = (M*MTSUN_SI/delta_t)*(np.gradient(phaseEcc))
    omegaImr = (M*MTSUN_SI/delta_t)*(np.gradient(phaseImr))

    if(tfreq_Hinsp(eta,e0,l0)>=0.0):
        print('t_freq>0')

    "Hider+ inspired"
    tjoin0 = tfreq_Hinsp(eta,e0,l0)
    tjoin = tjoin0 * M * MTSUN_SI
    fjoin = np.argmin(abs(tEcc-tjoin))

    "frequency model"
    tstop = min(tEcc[-1],-30*M*MTSUN_SI)
    lst = np.argmin(abs(tEcc-tstop))

    indx = lst - fjoin
    a0 = []
    n = indx - 1 
    k = 0
    for i in range(fjoin,fjoin+indx):
        a0.append(((n-k)*omegaEcc[i]+k*omegaImr[i])/n)
        k = k+1

    f1 = np.concatenate((omegaEcc[0:fjoin],a0))
    frequency_model = np.concatenate((f1,omegaImr[fjoin+indx:len(omegaImr)]))
    phase_f_model = np.cumsum(frequency_model)/(M*MTSUN_SI/delta_t)

    hp_f_model = h22amp_model * np.cos(phase_f_model)
    hc_f_model = h22amp_model * np.sin(phase_f_model)
    
    return np.array(hp_f_model), np.array(hc_f_model), np.array(t_model)

### $(\ell, m) = (5, 5)$ mode waveform

In [ ]:
def inspiral_55(M0,q,e0,l0,flow,inc,d0,delta_t):
    eta=neu=nu=q/(1+q)**2 
    #xref=0.075  #reference frequency
    #xlow=0.045  #initial frequency
    G=c=M=d=1
    M2=M/(1+q)
    M1=M2*q
    Delta=math.sqrt(1-(4*neu))
    eta=nu=neu
    gamma=EulerGamma= 0.577215664901

    conv=M*MTSUN_SI              #total mass (needed to convert PN time t to t/M)
    M_SI = M * MSUN_SI
    D_SI = 10**6 * PC_SI * d
  
    xlow = ((M0*MTSUN_SI*math.pi*flow)**(2/3))
    f_low =(xlow**(3/2)/(M*MTSUN_SI*math.pi)) #Lower frequency of EccTD wf which is in our hands
    #f_ref=freq(xref)            #Ref frequency of EccTD wf
    ####################################################
    
    #if l0<0:
        #l0 = l0 + 2*np.pi
    
    #Calculating ecc at xlow for generating EccTD wf-->eccentricity evolution Eq. (4.17a,b) Moore et. al.(2016)

    #%run GW_functions_model.ipynb         #Notebook with all PN evolution equations   
    x=xlow                          #x=point where eccentricity is required
    v=math.sqrt(x)
    #vref=math.sqrt(xref)
    xie=v**3
    #xiref=vref**3

    #Gives e at xie with b.c.--> e=eref when xie=xiref
    #e=eref*(xiref/xie)**(19/18)*epsilon(xie, eta)/epsilon(xiref, eta)  #eccentricity evolution 
    
    if delta_t>=1/2**14:
        del_t = 1/2**14
    elif delta_t<1/2**14 and delta_t>=1/2**16:
        del_t = 1/2**16
    elif delta_t<1/2**16 and delta_t>=1/2**18:
        del_t = 1/2**18
    else:
        del_t = 1/2**20
    
    phase_EccTD, tVec_PN = PNparams(M,q,d,f_low,e0,del_t)
    
    tC_NR = 0
    #xi_ref=xref**(3/2)
    #v_ref=xi_ref**(1/3)
    #input initial params
    x0=xlow
    xi0=x0**(3/2)
    v0=xi0**(1/3)
    #e0=(eref*(xi_ref/xi0)**(19/18))*(epsilon(xi0, eta)/epsilon(xi_ref, eta))
    #l0=mean_anomaly(xi_ref, xi0, lref, eta, e0)
    theta=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN)**(-1/8)
    theta0=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN[0])**(-1/8)
    fVec=x_from_t(theta, theta0, e0, M, eta)
    
    plotIdx2=np.nonzero(fVec>=0)
    fVec=fVec[plotIdx2]
    
    xiVec=(np.pi*M*fVec)
    xVec=xiVec**(2/3)
    vVec=xiVec**(1/3)
    maxPNidx=len(xVec)
    tVec_PN=tVec_PN[:maxPNidx]
    
    lp=5
    mp=5
    
    j=0
    #omega=[]
    h55=[]
    h5_5=[]
    for i in xVec:
        v=math.sqrt(i)
        v0=math.sqrt(x0)
        xie=v**3
        xi0=v0**3
        l=mean_anomaly(xie, xi0, l0, eta, e0)
        e=e0*(xi0/xie)**(19/18)*epsilon(xie, eta)/epsilon(xi0, eta)
        #psi=phase_TT2(v, v0, phiC, eta)+W(xie, xi0, eta, e0, l)
        psi=phase_EccTD[j]
        j=j+1
        xi=l   #use xi for amplitude (xie is being used for v**3)
        x=i
        h=amplitude_55(xi,x,nu,e,Delta)  #check GW_functions.ipynb for the respective mode amplitude functions 
                                   #and the required function arguements
        hlm=8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,-1)*mp*psi/2))/d
        hl_m = 8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,+1)*mp*psi/2))/d

        h55.append(hlm)
        h5_5.append(hl_m)
        
    conv_t = M0*MTSUN_SI
    conv_h = G_SI*M0*MSUN_SI/(10**6 * PC_SI * d0)/C_SI/C_SI

    sph55, sph5_5 = sph_harmonics(inc,lp)
        
    h = np.multiply(h55,sph55) + np.multiply(h5_5,sph5_5)
    hp = np.real(h)
    hc = np.imag(h)
        
    hp = np.array(hp) * conv_h
    hc = np.array(hc) * conv_h
    time = tVec_PN * conv_t
    
    hp_intrp = interp1d(time, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(time, hc, kind='cubic',fill_value='extrapolate')
    t_intrp=np.arange(time[0], time[-1], delta_t)
    hp_intrp = hp_intrp(t_intrp)
    hc_intrp = hc_intrp(t_intrp)
    
    return np.array(hp_intrp), np.array(hc_intrp), np.array(t_intrp)

In [ ]:
def model_55(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    #m = total mass in units of solar mass
    #q0 = mass ratio
    #e0 = initial eccentricity
    #l0 = initial mean anomaly (-PI to PI)
    #fmin = initial frequency in units of Hz
    #angle = orbit inclination
    #d = distanve in units of MPc
    #delta_t = sampling rate
    
    M=m
    M1 = q0*M/(1+q0)
    M2 = M/(1+q0)
    eta = q0/(1+q0)**2
    #q0 = q
    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    

    hp, hc, tinsp = inspiral_55(M,q0,e0,l0,fmin,angle,d,delta_t)
    sp, sc = get_td_waveform(approximant='SEOBNRv4HM', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin, mode_array=[[5,5]],inclination=angle)


    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI
    tmin = max(tinsp[0]-tshift,sp.sample_times[0])

    #circular IMR
    sp_intrp = interp1d(sp.sample_times, sp, kind='cubic',fill_value='extrapolate')
    sc_intrp = interp1d(sc.sample_times, sc, kind='cubic',fill_value='extrapolate')
    tImr_intrp=np.arange(tmin, sp.sample_times[-1], delta_t)
    sp_intrp = sp_intrp(tImr_intrp)
    sc_intrp = sc_intrp(tImr_intrp)
    tImr = tImr_intrp
    hpImr = sp_intrp
    hcImr = sc_intrp
    h22Imr = hpImr + 1j*hcImr


    "time shift"
    "Hinder+ inspired"
    tshift = - tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI

    # interpolation EccentricTD
    hp_intrp = interp1d(tinsp-tshift, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(tinsp-tshift, hc, kind='cubic',fill_value='extrapolate')
    # new (unifrom) sampling rate (for interpolation and further processing)
    tEcc_intrp=np.arange(tmin, tinsp[-1]-tshift, delta_t)
    hp_intrp = hp_intrp(tEcc_intrp)
    hc_intrp = hc_intrp(tEcc_intrp)
    tEcc = tEcc_intrp
    hpEcc = hp_intrp
    hcEcc = hc_intrp
    h22Ecc = hpEcc + 1j*hcEcc

    phaseEcc = np.unwrap(np.angle(h22Ecc)*2)/2
    phaseImr = -np.unwrap(np.angle(h22Imr)*2)/2
    dphase = phaseEcc[0] - phaseImr[0]
    hp_new = real(h22Ecc * exp(-1j * dphase))
    hc_new = imag(h22Ecc * exp(-1j * dphase))

    phase_new = np.unwrap(np.angle(hp_new+1j*hc_new)*2)/2

    phaseEcc = phase_new
    h22Ecc_new = hp_new+1j*hc_new

    if(tamp_Hinsp(eta,e0,l0)>=0.0):
        print('t_amp>0')

    "Hinder+ inspired"
    arg = np.argmin(abs(tEcc - tamp_Hinsp(eta,e0,l0)*M*MTSUN_SI))
    Idxjoin = arg 

    "Hinder+ inspired"
    t_amp = tEcc[Idxjoin] - 500*M*MTSUN_SI
    idxstr = np.argmin(abs(tEcc-t_amp))

    "amplitude model"
    amp = []
    count = 0
    length = Idxjoin - idxstr 

    for i in range(idxstr,Idxjoin):
        amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
        count = count+1

    t_model = np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))
    h22amp = np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
    h22amp_model = np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))

    omegaEcc = (M*MTSUN_SI/delta_t)*(np.gradient(phaseEcc))
    omegaImr = (M*MTSUN_SI/delta_t)*(np.gradient(phaseImr))

    if(tfreq_Hinsp(eta,e0,l0)>=0.0):
        print('t_freq>0')

    "Hider+ inspired"
    tjoin0 = tfreq_Hinsp(eta,e0,l0)
    tjoin = tjoin0 * M * MTSUN_SI
    fjoin = np.argmin(abs(tEcc-tjoin))

    "frequency model"
    tstop = min(tEcc[-1],-30*M*MTSUN_SI)
    lst = np.argmin(abs(tEcc-tstop))

    indx = lst - fjoin
    a0 = []
    n = indx - 1 
    k = 0
    for i in range(fjoin,fjoin+indx):
        a0.append(((n-k)*omegaEcc[i]+k*omegaImr[i])/n)
        k = k+1

    f1 = np.concatenate((omegaEcc[0:fjoin],a0))
    frequency_model = np.concatenate((f1,omegaImr[fjoin+indx:len(omegaImr)]))
    phase_f_model = np.cumsum(frequency_model)/(M*MTSUN_SI/delta_t)

    hp_f_model = h22amp_model * np.cos(phase_f_model)
    hc_f_model = h22amp_model * np.sin(phase_f_model)
    
    return np.array(hp_f_model), np.array(hc_f_model), np.array(t_model)